In [3]:
!git clone https://github.com/Paperspace/DataAugmentationForObjectDetection


fatal: destination path 'DataAugmentationForObjectDetection' already exists and is not an empty directory.


In [1]:
%cd DataAugmentationForObjectDetection

/Users/sbhasin/DataAugmentationForObjectDetection


In [1]:
!pwd

/Users/sbhasin


In [2]:
from data_aug.data_aug import *
from data_aug.bbox_util import *
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
import pickle as pkl

In [5]:
%cd desktop

/Users/sbhasin/Desktop


In [6]:
%cd logo-detection/data/dataset

/Users/sbhasin/Desktop/logo-detection/data/dataset


In [7]:
a_file = open("train.txt", "r")

list_of_lists = []
for line in a_file:
    list_of_lists.append(line[:line.find("\n")])

a_file.close()

print(list_of_lists)
def readFile(fileName):
        fileObj = open(fileName, "r") #opens the file in read mode
        words = fileObj.read().splitlines() #puts the file into an array
        fileObj.close()
        return words
def writefile(filename, array):
        array = array.tolist()
        f = open(filename, 'w+')
        rando = []
        for i in array:
            m = 0
            string = ''
            z= [0 for i in range(5)]
            z[1:] = i[:4]
            z[0] = int(i[-1])
            for x in z:
                string += str(x)
                if m!=len(z)-1:
                    string+= ' '
                else:
                    string+='\n'
                m+=1 
            rando.append(string)
        f.writelines(rando)
        f.close()
def write (filename, array):
    array = array.tolist()
    f = open(filename, 'w+')
    rando = []
    for z in array:
        m = 0
        string = ''
        for x in z:
                string += str(x)
                if m!=len(z)-1:
                    string+= ' '
                else:
                    string+='\n'
                m+=1 
        rando.append(string)
    f.writelines(rando)
    f.close()

        

['/porsche/img000047_54.jpg', '/porsche/img000055_54.jpg', '/porsche/img000063_54.jpg', '/porsche/img000049_54.jpg', '/porsche/img000061_54.jpg', '/porsche/img000006_54.jpg', '/porsche/img000077_54.jpg', '/porsche/img000053_54.jpg', '/porsche/img000086_54.jpg', '/porsche/img000070_54.jpg', '/porsche/img000058_54.jpg', '/porsche/img000054_54.jpg', '/porsche/img000048_54.jpg', '/porsche/img000085_54.jpg', '/porsche/img000019_54.jpg', '/porsche/img000076_54.jpg', '/porsche/img000050_54.jpg', '/porsche/img000074_54.jpg', '/porsche/img000078_54.jpg', '/intel/img000130_35.jpg', '/intel/img000030_35.jpg', '/intel/img000053_35.jpg', '/intel/img000122_35.jpg', '/intel/img000141_35.jpg', '/intel/img000014_35.jpg', '/intel/img000177_35.jpg', '/intel/img000106_35.jpg', '/intel/img000006_35.jpg', '/intel/img000118_35.jpg', '/intel/img000020_35.jpg', '/intel/img000120_35.jpg', '/intel/img000051_35.jpg', '/intel/img000167_35.jpg', '/intel/img000179_35.jpg', '/intel/img000004_35.jpg', '/intel/img00010

In [10]:
import numpy as np
brands = {}
count = {}
filenames = []
for i in list_of_lists:
    y = []
    j = 0
    for x in i:
        if x=='/':
            y.append(j)
        j+=1 
    brand = i[y[-2]+1:y[-1]]
    brands[brand] = brands.get(brand,0) + 1 
    filename = i[y[-1]+1:]
    filenames.append(filename)

print(filenames)
def normalise (array):
    array1 = array.copy()
    dw = 1./416
    dh = 1./416
    booly = False 
    returnable  = -1  
    new = array1[:,-1]
    array1 = array1[:,:4]
    width = array1[:,2] - array1[:,0]
    height = array1[:,3] - array1[:,1]
    array1[:,0] = (((array1[:,0] + array1[:,2])/2) - 1)*dw
    array1[:,1] = (((array1[:,1] + array1[:,3])/2) - 1)*dh 
    array1[:,2] = width*dw
    array1[:,3] = height*dh
    array1 = np.around(array1, decimals = 10)
    initial = np.zeros((array1.shape[0],5))
    initial[:,-1] = new
    initial[:,:4] = array1
    for i in initial:
        k = 1
        for j in i:
            if j>=1 and k!= 5:
                booly = True
                returnable = j
                break
            k+=1     
    return initial, booly, returnable  


['img000047_54.jpg', 'img000055_54.jpg', 'img000063_54.jpg', 'img000049_54.jpg', 'img000061_54.jpg', 'img000006_54.jpg', 'img000077_54.jpg', 'img000053_54.jpg', 'img000086_54.jpg', 'img000070_54.jpg', 'img000058_54.jpg', 'img000054_54.jpg', 'img000048_54.jpg', 'img000085_54.jpg', 'img000019_54.jpg', 'img000076_54.jpg', 'img000050_54.jpg', 'img000074_54.jpg', 'img000078_54.jpg', 'img000130_35.jpg', 'img000030_35.jpg', 'img000053_35.jpg', 'img000122_35.jpg', 'img000141_35.jpg', 'img000014_35.jpg', 'img000177_35.jpg', 'img000106_35.jpg', 'img000006_35.jpg', 'img000118_35.jpg', 'img000020_35.jpg', 'img000120_35.jpg', 'img000051_35.jpg', 'img000167_35.jpg', 'img000179_35.jpg', 'img000004_35.jpg', 'img000108_35.jpg', 'img000116_35.jpg', 'img000098_35.jpg', 'img000094_35.jpg', 'img000171_35.jpg', 'img000012_35.jpg', 'img000100_35.jpg', 'img000000_35.jpg', 'img000155_35.jpg', 'img000124_35.jpg', 'img000090_35.jpg', 'img000082_35.jpg', 'img000180_35.jpg', 'img000080_35.jpg', 'img000002_35.jpg',

In [11]:
def augmentor (values, no, imgname, final_img, final_bboxes):
    if len(values)!=0:
        rot = values[0]
        shear = values[1]
        img_v1, bboxes_v1 = Sequence([RandomHorizontalFlip(),RandomTranslate(), RandomRotate(rot),
        RandomShear(shear)])(final_img.copy(),final_bboxes.copy())
        img_v1 = cv2.cvtColor(img_v1, cv2.COLOR_BGR2GRAY)
        bboxes_v11, booly, returnable  = normalise(bboxes_v1)
        if booly is not True:
            cv2.imwrite(os.path.join('train/images',imgname +'v'+ no +'.jpg'),img_v1)
            writefile(os.path.join('train/labels',imgname +'v'+ no +'.txt'),bboxes_v11)
            #cv2.imwrite(os.path.join(imagepat,'images',imgname +'v'+ no +'.jpg'),img_v1)
            #write(os.path.join(imagepat,'labels',imgname +'v'+ no +'.txt'),bboxes_v1)
        else:
            print('EXCEEDED', imgname, returnable )
                
    else:
        img_v1 = cv2.cvtColor(final_img, cv2.COLOR_BGR2GRAY) 
        final_bboxes1, booly, returnable = normalise(final_bboxes)
        
        if booly is not True:
            cv2.imwrite(os.path.join('images',imgname +'v1.jpg'),img_v1)
            writefile(os.path.join('labels',imgname +'v1.txt'),final_bboxes1)
            #cv2.imwrite(os.path.join(imagepat,'images',imgname +'v1.jpg'),img_v1)
            #write(os.path.join(imagepat,'labels',imgname +'v1.txt'),final_bboxes)
        else:
            print('EXCEEDED', imgname, returnable )

        
        

In [12]:
for i in list_of_lists:
    
    y = []
    j = 0
    for x in i:
        if x=='/':
            y.append(j)
        j+=1 
    brand = i[y[-2]+1:y[-1]]
    imgname = i[y[-1]+1:-4]
    imagepath = i[y[-1]+1:] 
    textpath = imagepath.replace('jpg', 'txt')
    img = cv2.imread(imagepath)[:,:,::-1]
    annots = readFile(textpath)
    bboxes = []
    for i in annots:
          z = [0 for i in range(5)]
          y = [float(j) for j in i.split()]
          z[:4] = y[1:]
          z[-1] = y[0]
          bboxes.append(z)
    bboxes = np.array(bboxes)
    img_resize, final_bboxes = Resize(416)(img.copy(), bboxes.copy())
    final_img = img_resize 
    print("image shape", img.shape, "path", imgname)
    if brands[brand]<50:
        try:
            augmentor([30,0.4], '1',  imgname, final_img, final_bboxes)
            augmentor([45,0.6], '2',  imgname, final_img, final_bboxes)
            augmentor([40,0.5], '3',  imgname, final_img, final_bboxes)
            augmentor([50,0.5], '4',  imgname, final_img, final_bboxes)
            augmentor([20,0.3], '5',  imgname, final_img, final_bboxes)
            augmentor([60,0.6], '6',  imgname, final_img, final_bboxes)
            os.remove(imagepath)
            os.remove(textpath)
        except IndexError:
            augmentor([], '1', imgname, final_img, final_bboxes)
            os.remove(imagepath)
            os.remove(textpath)
            print('indexerror')        

    elif brands[brand]<100:
        try:
            augmentor([30,0.4], '1', imgname, final_img, final_bboxes)
            augmentor([60,0.6], '2',  imgname, final_img, final_bboxes)
            augmentor([50,0.5], '3',  imgname, final_img, final_bboxes)
            augmentor([20,0.3], '4',  imgname, final_img, final_bboxes)
            os.remove(imagepath)
            os.remove(textpath)
        except IndexError:
            augmentor([], '1', imgname, final_img, final_bboxes)
            os.remove(imagepath)
            os.remove(textpath)
            print('indexerror')
    else:
        try:
            augmentor([45,0.4], '1', imgname, final_img, final_bboxes)
            augmentor([60,0.3], '2',  imgname, final_img, final_bboxes)
            os.remove(imagepath)
            os.remove(textpath)
        except IndexError:
            augmentor([], '1',  imgname, final_img, final_bboxes)
            os.remove(imagepath)
            os.remove(textpath)
            print('indexerror')
    
    
    
    

    
    
    
    

    
    

image shape (810, 1280, 3) path img000047_54
image shape (394, 700, 3) path img000055_54
image shape (960, 1600, 3) path img000063_54
image shape (318, 646, 3) path img000049_54
image shape (408, 600, 3) path img000061_54
image shape (1024, 819, 3) path img000006_54
image shape (927, 1253, 3) path img000077_54
image shape (607, 997, 3) path img000053_54
image shape (612, 1000, 3) path img000086_54
image shape (615, 1600, 3) path img000070_54
image shape (605, 996, 3) path img000058_54
image shape (482, 1000, 3) path img000054_54
image shape (683, 1024, 3) path img000048_54
image shape (681, 1024, 3) path img000085_54
image shape (350, 620, 3) path img000019_54
image shape (267, 400, 3) path img000076_54
image shape (389, 690, 3) path img000050_54
image shape (363, 1000, 3) path img000074_54
image shape (960, 1600, 3) path img000078_54
image shape (270, 480, 3) path img000130_35
image shape (1703, 1181, 3) path img000030_35
image shape (3200, 2400, 3) path img000053_35
indexerror
image 

In [19]:
import os
import shutil
j = 0

for i in filenames:
    j+=1 
    if j%4==0:
        filename = i[:-4]  + 'v1' + '.jpg'
        textname = i[:-4]  + 'v1' + '.txt'
        dest = "valid/images/" + filename
        textdest = "valid/labels/" + textname
        source = "images/" + filename
        textsource = "labels/" + textname
        x = shutil.copy(source, dest)
        y = shutil.copy(textsource, textdest)
        os.remove(source)
        os.remove(textsource)
    else:
        continue 
    

    


1025


In [18]:
!pwd

/Users/sbhasin/Desktop/logo-detection/data/dataset


In [15]:
print(len(filenames))

4100
